In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import json
import ta
import matplotlib.pyplot as pl
import plotly.express as px
import plotly.graph_objects as go


In [4]:
df = pd.read_csv('stocks_data.csv', index_col=0)
df

,Date,AZO,BKNG,CMG,NVR
0,2013-01-02,356.880005,635.308105,6.021200,930.130005
1,2013-01-03,356.140015,645.588013,6.019000,949.000000
2,2013-01-04,360.850006,645.269592,6.003600,954.489990
3,2013-01-07,356.170013,654.196167,5.991800,966.890015
4,2013-01-08,348.250000,654.235962,5.955200,974.500000
...,...,...,...,...,...
2574,2023-03-24,2329.399902,2487.225098,32.485001,5413.000000
2575,2023-03-27,2368.550049,2496.161865,32.956402,5375.450195
2576,2023-03-28,2405.219971,2532.793701,33.057999,5413.560059
2577,2023-03-29,2395.370117,2559.314453,33.134399,5433.990234


In [6]:
df1 = pd.melt(df, id_vars='Date')
df1

,Date,variable,value
0,2013-01-02,AZO,356.880005
1,2013-01-03,AZO,356.140015
2,2013-01-04,AZO,360.850006
3,2013-01-07,AZO,356.170013
4,2013-01-08,AZO,348.250000
...,...,...,...
10311,2023-03-24,NVR,5413.000000
10312,2023-03-27,NVR,5375.450195
10313,2023-03-28,NVR,5413.560059
10314,2023-03-29,NVR,5433.990234


In [7]:
# plot the 4 stocks together 


df1 = pd.melt(df, id_vars='Date')
px.line(df1, x = 'Date', y = 'value', color='variable')

In [8]:
df.head(2)

,Date,AZO,BKNG,CMG,NVR
0,2013-01-02,356.880005,635.308105,6.0212,930.130005
1,2013-01-03,356.140015,645.588013,6.0190,949.000000


**Visualization: AZO**

In [9]:
# Line Chart
fig = go.Figure(data=go.Scatter(x=df['Date'], y=df['AZO'], mode='lines', name='Stock Price'))
fig.update_layout(title='Line Chart of Stock Prices', xaxis_title='Date', yaxis_title='Price')
fig.show()

**Moving Averages : AZO** 

In [10]:
# Calculating moving averages 
# Stock : AZO

df['MA20'] = df['AZO'].rolling(window=20).mean()
df['MA50'] = df['AZO'].rolling(window=50).mean()

# Moving Averages Chart
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'], y=df['AZO'], mode='lines', name='Stock Price'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['MA20'], mode='lines', name='20-Day MA'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['MA50'], mode='lines', name='50-Day MA'))
fig.update_layout(title='Moving Averages', xaxis_title='Date', yaxis_title='Price')
fig.show()

**Bollinger Bands: AZO**

In [12]:
# Calculating Bollinger Bands
df['MA20'] = df['AZO'].rolling(window=20).mean()
df['BB_up'] = df['MA20'] + 2 * df['AZO'].rolling(window=20).std()
df['BB_dn'] = df['MA20'] - 2 * df['AZO'].rolling(window=20).std()

# Bollinger Bands Chart
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'], y=df['AZO'], mode='lines', name='Stock Price'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['BB_up'], mode='lines', name='Upper Band'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['BB_dn'], mode='lines', name='Lower Band'))
fig.update_layout(title='Bollinger Bands', xaxis_title='Date', yaxis_title='Price')
fig.show()

**RSI :AZO**

In [13]:
def RSI(series, period=14):
    delta = series.diff(1)
    gain = (delta.where(delta > 0, 0)).fillna(0)
    loss = (-delta.where(delta < 0, 0)).fillna(0)
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [14]:
df['RSI'] = RSI(df['AZO'])
# RSI Chart
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'], y=df['RSI'], mode='lines', name='RSI'))
fig.update_layout(title='Relative Strength Index (RSI)', xaxis_title='Date', yaxis_title='RSI')
fig.show()

**MACD :AZO**

In [15]:
# Calculating MACD
df['EMA12'] = df['AZO'].ewm(span=12, adjust=False).mean()
df['EMA26'] = df['AZO'].ewm(span=26, adjust=False).mean()
df['MACD'] = df['EMA12'] - df['EMA26']
df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()

# MACD Chart
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['Date'], y=df['MACD'], mode='lines', name='MACD'))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Signal'], mode='lines', name='Signal Line'))
fig.update_layout(title='MACD', xaxis_title='Date', yaxis_title='Value')
fig.show()

**Correlation between Stocks**

In [17]:
df_corr = df[['AZO', 'BKNG', 'CMG', 'NVR']]

In [18]:
corr = df_corr.corr()

# Heatmap
fig = px.imshow(corr, text_auto=True, title='Stock Correlation Heatmap')
fig.show()

**BKNG**